<a href="https://colab.research.google.com/github/clustering-jun/KMU-Data_Science/blob/main/Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Linear Regression Practice**


## **PyTorch Basic**

In [1]:
import torch

### **Tensor**
파이토치에서 사용하는 가장 기본적인 자료구조
- 다양한 수식 계산 지원
- 수학의 스칼라, 벡터, 행렬 등을 일반화한 개념
 - Scalar, Vector, Matrix, Tensor

In [10]:
x = torch.tensor([[1,2,3], [4,5,6], [7,8,9]])
y = torch.FloatTensor([[1,2,3], [4,5,6], [7,8,9]])

print(x)
print(x.type())

print(y)
print(y.type())

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])
torch.LongTensor
tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])
torch.FloatTensor


### **크기와 랭크(차원) 확인하기**
- size()
- shape()
- ndimension()

In [17]:
x = torch.FloatTensor([[1,2,3],[4,5,6],[7,8,9]])
y = torch.FloatTensor([[[1,2], [3,4]],[[1,2], [3,4]],[[1,2], [3,4]]])

print(x.size())
print(x.shape) # 더 자주 사용
print(x.ndimension()) # 2차원
print(y.ndimension()) # 3차원

print(len(y.shape))

torch.Size([3, 3])
torch.Size([3, 3])
2
3
3


### **텐서 모양 바꾸기**
* unsqueeze(x,i): tensor x에 i번째 차원 추가
* squeeze(x): tensor x에서 크기가 1인 차원 제거
* x.view([shape]): x를 [shape]의 모양으로 변환

In [ ]:
x = torch.FloatTensor([[[1,2], [3,4]],[[1,2], [3,4]],[[1,2], [3,4]]])

print(x.shape) # torch.Size([3, 2, 2])
print(x)

x0 = x.unsqueeze(0)
print(x0)
print(x0.shape) # torch.Size([1, 3, 2, 2])

x1 = x0.squeeze() # or: torch.squeeze(x0)
print(x1.shape) # torch.Size([3, 2, 2])

torch.Size([3, 2, 2])
tensor([[[1., 2.],
         [3., 4.]],

        [[1., 2.],
         [3., 4.]],

        [[1., 2.],
         [3., 4.]]])
tensor([[[[1., 2.],
          [3., 4.]],

         [[1., 2.],
          [3., 4.]],

         [[1., 2.],
          [3., 4.]]]])
torch.Size([1, 3, 2, 2])
torch.Size([3, 2, 2])


tensor([[1., 2., 3., 4.],
        [1., 2., 3., 4.],
        [1., 2., 3., 4.]])

In [32]:
x = torch.FloatTensor([[[1,2], [3,4]],[[1,2], [3,4]],[[1,2], [3,4]]])

print(x.view(12))
print(x.view(2,3,2))
print(x.view(3,-1))

tensor([1., 2., 3., 4., 1., 2., 3., 4., 1., 2., 3., 4.])
tensor([[[1., 2.],
         [3., 4.],
         [1., 2.]],

        [[3., 4.],
         [1., 2.],
         [3., 4.]]])
tensor([[1., 2., 3., 4.],
        [1., 2., 3., 4.],
        [1., 2., 3., 4.]])


### **행렬 연산**
- wx + b
- randn: 정규분포(Normal Distribution, 평균 0, 표준편차 1) 를 따르는 랜덤 값을 가지는 텐서 생성

In [ ]:
x = torch.FloatTensor([[1,2],[3,4],[5,6]])  # 3 x 2
w = torch.randn(1,2, dtype=torch.float) # 1 x 2  ==> 2 x 1
b = torch.randn(3,1, dtype=torch.float)

print(x @ w.T + b)
# torch.mm(x, w.T) --> matrix multiplication
# torch.t(w) == w.T
# x @ w.T

tensor([[ -6.1258],
        [-12.7275],
        [-17.1558]])


- Autograd: 기울기 계산
 - requires_grad 값이 설정된 tensor에 대해서, pytorch는 자동으로 기울기를 계산해줌.

In [34]:
w = torch.tensor(4.0, requires_grad=True)
a = w*3
l = a**2
l.backward()
print(w.grad)

tensor(72.)


- 참고

 - **연산 추적 시작 (requires_grad=True)**
   - 텐서가 `requires_grad=True`이면, 그 텐서에 대해 이루어진 모든 연산이 기록된다. 이때 PyTorch는 자동으로 각 연산에 대한 Function 객체를 생성한다.
  - **Computation Graph구축**
   - 연산이 일어날 때마다 PyTorch는 각 결과 텐서에 grad_fn (gradient function) 속성을 부여한다.
 - **역전파 (backward) 호출 시**
   - `.backward()`를 호출하면 PyTorch는 출력 노드부터 시작해 계산 그래프를 거꾸로 따라가며 체인 룰(chain rule) 로 미분을 계산한다.
 - **기울기 저장**
   - 모든 텐서 중 `requires_grad=True`로 설정된 텐서는 .grad 속성에 그에 대한 미분 결과(기울기) 가 저장된다.


## **Linear Regression with Pytorch**

In [37]:
import torch

# Find this function: y = 2*x1 + 3*x2 - 4
x_train = torch.FloatTensor([[1,2], [3,2], [3,7], [1,1], [1,0]])
y_train = torch.FloatTensor([[4], [8], [23], [1], [-2]])

w = torch.randn(2,1)
b = torch.randn(1)
lr = 0.01

 # w, b값 찾기
for epoch in range(3000):
  w.requires_grad_(True)
  b.requires_grad_(True) # 이렇게 안에 넣어야하는 이유??

  h = x_train @ w + b
  cost = ((h - y_train)**2).mean()

  cost.backward()

  # 경사하강법 (미분x)
  with torch.no_grad():
    w = w - lr * w.grad
    b = b - lr * b.grad

    if epoch % 100 == 0:
      print(epoch, cost.item(), w.squeeze(), b)

0 51.93864822387695 tensor([0.0564, 1.5065]) tensor([1.3564])
100 2.5828635692596436 tensor([0.1448, 3.2480]) tensor([-0.5878])
200 1.2982046604156494 tensor([0.5731, 3.2483]) tensor([-1.6208])
300 0.6667962074279785 tensor([0.9661, 3.1860]) tensor([-2.3020])
400 0.3426740765571594 tensor([1.2576, 3.1342]) tensor([-2.7836])
500 0.17610636353492737 tensor([1.4676, 3.0963]) tensor([-3.1281])
600 0.0905044674873352 tensor([1.6183, 3.0691]) tensor([-3.3750])
700 0.046511828899383545 tensor([1.7264, 3.0495]) tensor([-3.5519])
800 0.023903314024209976 tensor([1.8039, 3.0355]) tensor([-3.6788])
900 0.0122844148427248 tensor([1.8594, 3.0255]) tensor([-3.7697])
1000 0.00631310697644949 tensor([1.8992, 3.0182]) tensor([-3.8349])
1100 0.003244437975808978 tensor([1.9277, 3.0131]) tensor([-3.8817])
1200 0.0016673619393259287 tensor([1.9482, 3.0094]) tensor([-3.9152])
1300 0.0008568911580368876 tensor([1.9629, 3.0067]) tensor([-3.9392])
1400 0.0004403713974170387 tensor([1.9734, 3.0048]) tensor([-3

In [39]:
# 예측하기
x_test = torch.FloatTensor([[5,10], [2,7]])
y_test = x_test @ w + b
print(y_test) # 약 36, 21

tensor([[35.9998],
        [21.0001]])


## **Linear Regression with Scikit-learn**

### Python의 대표 머신러닝 라이브러리

In [41]:
from sklearn.linear_model import LinearRegression

x = [[1,2], [3,2], [3,7], [1,1], [1,0]]
y = [[4], [8], [23], [1], [-2]]

lr = LinearRegression() # 모델 생성
lr.fit(x, y) # 학습 (피팅)

print(lr.coef_, lr.intercept_)

[[2. 3.]] [-4.]


In [42]:
# 예측하기
x_test = [[5,10]]

y_test = lr.predict(x_test)
print(y_test)

[[36.]]
